In [10]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("hsankesara/flickr-image-dataset")

print("Path to dataset files:", path)

Download already complete (8765396518 bytes).
Extracting files...


OSError: [Errno 28] No space left on device

In [7]:
import numpy as np
from PIL import Image
import os
import string
from pickle import dump
from pickle import load
from keras.applications.xception import Xception
from keras.applications.xception import preprocess_input
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from tensorflow.keras.layers import Add
from keras.models import Model, load_model
from keras.layers import Input, Dense #Keras to build our CNN and LSTM
from keras.layers import LSTM, Embedding, Dropout
from tqdm.notebook import tqdm # to check loop progress
tqdm().pandas()

0it [00:00, ?it/s]

In [11]:
from kaggle.api.kaggle_api_extended import KaggleApi
import zipfile, os

os.environ['KAGGLE_CONFIG_DIR'] = '/Users/Nanen/Documents/academic/personal-projects/caption-generator'

api = KaggleApi()
api.authenticate()

# Example: Titanic dataset
dataset = 'competitions/titanic'  # or 'zynicide/wine-reviews' for datasets
download_path = '/tmp/titanic'

api.competition_download_files(dataset, path=download_path)

# Unzip if needed
with zipfile.ZipFile(f"{download_path}/titanic.zip", 'r') as zip_ref:
    zip_ref.extractall(download_path)

print(os.listdir(download_path))

OSError: Could not find kaggle.json. Make sure it's located in /Users/Nanen/.kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/

DATA CLEANING

In [ ]:
# create a dictionary that maps images with all 5 captions
def mapImgWithCaptions(filename):
    file = open(filename, "r")
    text = file.read()
    captions = text.split('n')
    descriptions = {}
    for caption in captions[:-1]:
        img, caption = caption.split('t')
        if img[:-2] not in descriptions:
            descriptions[img[:-2]] = [caption]
        else:
            descriptions[img[:-2]].append(caption)
    return descriptions


# clean the data by taking all descriptions as input. 
# uppercase to lowercase conversion, punctuation removal, and removal of the number containing words.
def txt_clean(captions):
    table = str.maketrans('', '', string.punctuation)
    for img, caps in captions.items():
        for i, img_caption in enumerate(caps):
            img_caption.replace("-", " ")
            descp = img_caption.split()
            # uppercase to lowercase
            descp = [wrd.lower() for wrd in descp]
            # remove punctuation from each token
            descp = [wrd.translate(table) for wrd in descp]
            # remove hanging 's and a
            descp = [wrd for wrd in descp if (len(wrd) > 1)]
            # remove words containing number with them
            descp = [wrd for wrd in descp if (wrd.isalpha())]
            # convert back to strings
            img_caption = ' '.join(descp)
            captions[img][i] = img_caption
    return captions

dataset_text = " - Image Caption GeneratorFlickr_8k_text"
dataset_images = " - Image Caption GeneratorFlickr_8k_Datatset"
filename = dataset_text + "/" + "Flickr8k.token.txt"
descriptions = mapImgWithCaptions(filename)

In [ ]:
# extract the features from the model
model = Xception(include_top=False, pooling='avg')